In [65]:
import torch

In [66]:
import torch
from sympy.printing.tensorflow import tensorflow

In [67]:
pip install torch-geometric

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [68]:
from torch_geometric.data import Data

In [69]:
data=torch.load('graph_data_with_txt.pt')

C:\Users\Dell\AppData\Local\Temp\ipykernel_15844\951303083.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data=torch.load('graph_data_with_txt.pt')


In [70]:
#data.y[data.y == -1] = 3

In [71]:
# num_nodes = data.num_nodes
# num_train = int(0.8 * num_nodes)
# train_mask = torch.zeros(num_nodes, dtype=torch.bool)
# train_mask[:num_train] = 1

In [72]:
# data.train_mask = train_mask
# data.test_mask = ~train_mask

In [73]:
num_nodes = data.num_nodes
train_ratio, val_ratio = 0.8, 0.1  # 80% train, 10% val, 10% test

perm = torch.randperm(num_nodes)
train_size = int(train_ratio * num_nodes)
val_size = int(val_ratio * num_nodes)

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
val_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[perm[:train_size]] = True
val_mask[perm[train_size:train_size + val_size]] = True
test_mask[perm[train_size + val_size:]] = True

data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

In [74]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.nn.functional import dropout
from torch_geometric.nn import GCNConv, Linear, SAGEConv

In [75]:
import torch
from torch_geometric.nn import GCNConv
from torch.nn import Linear, Dropout, LeakyReLU


In [76]:
import torch
from torch.nn import Linear, Dropout, Sigmoid
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(-1, 64)
        self.conv2 = GCNConv(64, 128)
        self.conv3 = GCNConv(128, 256)
        self.conv4 = GCNConv(256, 512)
        self.linear = Linear(512, 256)
        self.linear1=Linear(256,128)
        self.linear2 = Linear(128, 64)
        self.linear3 = Linear(64, 1)
        self.dropout = Dropout(p=0.4)

    def forward(self, x, edge_index, edge_weight=None):
        x = self.conv1(x, edge_index, edge_weight=edge_weight)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv2(x, edge_index, edge_weight=edge_weight)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv3(x,edge_index, edge_weight=edge_weight)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv4(x,edge_index, edge_weight=edge_weight)
        x = torch.relu(x)
        x = self.dropout(x)
        
        # x = self.conv5(x,edge_index)
        # x = torch.relu(x)
        # x = self.dropout(x)
        x=self.linear(x)
        x=self.linear1(x)
        x=self.linear2(x)
        x = self.linear3(x)
        return x

In [77]:
import torch
from torch.nn import Linear, Dropout, Sigmoid
from torch_geometric.nn import SAGEConv
class SAGE(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SAGEConv(-1, 64)
        self.conv2 = SAGEConv(64, 128)
        self.conv3 = SAGEConv(128, 256)
        self.conv4 = SAGEConv(256, 512)
        self.linear = Linear(512, 256)
        self.linear1=Linear(256,128)
        self.linear2 = Linear(128, 64)
        self.linear3 = Linear(64, 1)
        self.dropout = Dropout(p=0.4)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv3(x,edge_index)
        x = torch.relu(x)
        x = self.dropout(x)
        
        x = self.conv4(x,edge_index)
        x = torch.relu(x)
        x = self.dropout(x)
        
        # x = self.conv5(x,edge_index)
        # x = torch.relu(x)
        # x = self.dropout(x)
        x=self.linear(x)
        x=self.linear1(x)
        x=self.linear2(x)
        x = self.linear3(x)
        return x

In [78]:
# def train(model, dataset, optimizer, criterion, epochs=5):
#     train_mask = dataset.train_mask
#     for epoch in range(epochs):
#         model.train()
#         optimizer.zero_grad()
#         out = model(dataset.x, dataset.edge_index,dataset.edge_weight).squeeze()  # Output shape: [num_nodes]
#         loss = criterion(out[train_mask], dataset.y[train_mask].float()) 
#         loss.backward()
#         optimizer.step()
#         print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

In [79]:
def train(model, dataset, optimizer, criterion, epochs=5):
    train_mask = dataset.train_mask
    val_mask = dataset.val_mask  # Assuming `val_mask` is available in the dataset
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        # Forward pass
        out = model(dataset.x, dataset.edge_index, dataset.edge_weight).squeeze()
        
        # Compute loss on the training set
        loss = criterion(out[train_mask], dataset.y[train_mask].float())
        loss.backward()
        optimizer.step()

        # Validation Performance
        model.eval()
        with torch.no_grad():
            val_out = out[val_mask]
            val_loss = criterion(val_out, dataset.y[val_mask].float())
            val_pred = (torch.sigmoid(val_out) > 0.5).long()
            # val_acc = accuracy_score(dataset.y[val_mask].cpu(), val_pred.cpu())
        
        print(f'Epoch: {epoch:03d}, Train Loss: {loss:.4f}, Val Loss: {val_loss:.4f}')

In [80]:
data

Data(x=[4150, 66], edge_index=[2, 439838], y=[4150], edge_weights=[0], train_mask=[4150], val_mask=[4150], test_mask=[4150])

In [100]:
model_gcn = GCN()

In [101]:
model_sage=SAGE()

In [102]:
model_gcn

GCN(
  (conv1): GCNConv(-1, 64)
  (conv2): GCNConv(64, 128)
  (conv3): GCNConv(128, 256)
  (conv4): GCNConv(256, 512)
  (linear): Linear(in_features=512, out_features=256, bias=True)
  (linear1): Linear(in_features=256, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=64, bias=True)
  (linear3): Linear(in_features=64, out_features=1, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
)

In [109]:
optimizer = Adam(model_gcn.parameters(), lr=0.001)

In [104]:
criterion = torch.nn.BCEWithLogitsLoss()

In [86]:
(data.y == 0).sum().item()

3064

In [107]:
train(model=model_sage, dataset=data,
      criterion=criterion, optimizer=optimizer,
      epochs=50)

TypeError: SAGE.forward() takes 3 positional arguments but 4 were given

In [110]:
train(model=model_gcn, dataset=data,
      criterion=criterion, optimizer=optimizer,
      epochs=50)

Epoch: 000, Train Loss: 0.5096, Val Loss: 0.5306
Epoch: 001, Train Loss: 0.5005, Val Loss: 0.5409
Epoch: 002, Train Loss: 0.5029, Val Loss: 0.5461
Epoch: 003, Train Loss: 0.5027, Val Loss: 0.5456
Epoch: 004, Train Loss: 0.4986, Val Loss: 0.5382
Epoch: 005, Train Loss: 0.5016, Val Loss: 0.5487
Epoch: 006, Train Loss: 0.5014, Val Loss: 0.5259
Epoch: 007, Train Loss: 0.5011, Val Loss: 0.5292
Epoch: 008, Train Loss: 0.5020, Val Loss: 0.5319
Epoch: 009, Train Loss: 0.4971, Val Loss: 0.5361
Epoch: 010, Train Loss: 0.4966, Val Loss: 0.5256
Epoch: 011, Train Loss: 0.4998, Val Loss: 0.5378
Epoch: 012, Train Loss: 0.4930, Val Loss: 0.5313
Epoch: 013, Train Loss: 0.4925, Val Loss: 0.5357
Epoch: 014, Train Loss: 0.4978, Val Loss: 0.5473
Epoch: 015, Train Loss: 0.4919, Val Loss: 0.5384
Epoch: 016, Train Loss: 0.4971, Val Loss: 0.5315
Epoch: 017, Train Loss: 0.4920, Val Loss: 0.5293
Epoch: 018, Train Loss: 0.4948, Val Loss: 0.5310
Epoch: 019, Train Loss: 0.4949, Val Loss: 0.5348
Epoch: 020, Train Lo

In [89]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [111]:
def test(model, dataset):
    model.eval()
    test_mask = dataset.test_mask

    with torch.no_grad():
        out = model(dataset.x, dataset.edge_index, dataset.edge_weight).squeeze()
        test_out = out[test_mask]
        test_pred = (torch.sigmoid(test_out) > 0.5).long()
        test_labels = dataset.y[test_mask]

    # Calculate metrics
    test_acc = accuracy_score(test_labels.cpu(), test_pred.cpu())
    test_precision = precision_score(test_labels.cpu(), test_pred.cpu(), average='macro')
    test_recall = recall_score(test_labels.cpu(), test_pred.cpu(), average='macro')
    test_f1 = f1_score(test_labels.cpu(), test_pred.cpu(), average='macro')

    print(f'Test Accuracy: {test_acc:}')
    print(f'Test Precision: {test_precision:}')
    print(f'Test Recall: {test_recall:}')
    print(f'Test F1 Score: {test_f1:}')

In [112]:
test(model=model_gcn, dataset=data)

Test Accuracy: 0.7614457831325301
Test Precision: 0.6880614314691393
Test Recall: 0.6167476164777838
Test F1 Score: 0.6289321808870946


In [178]:
from sklearn.cluster import KMeans

In [27]:
from sklearn.cluster import AgglomerativeClustering

# USING Class Weighting

In [92]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [113]:
labels = data.y.cpu().numpy() 
classes = np.unique(labels)

In [114]:
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=labels)

In [115]:
weights = torch.tensor(class_weights, dtype=torch.float) 

In [116]:
criterion2 = torch.nn.BCEWithLogitsLoss(pos_weight=weights[1])


In [117]:
model_gcn2 = GCN()
optimizer = Adam(model_gcn.parameters(), lr=0.001)

In [118]:
train(model=model_gcn2, dataset=data,
      criterion=criterion2, optimizer=optimizer,
      epochs=50)

Epoch: 000, Train Loss: 0.8515, Val Loss: 0.8629
Epoch: 001, Train Loss: 0.8511, Val Loss: 0.8630
Epoch: 002, Train Loss: 0.8517, Val Loss: 0.8622
Epoch: 003, Train Loss: 0.8504, Val Loss: 0.8625
Epoch: 004, Train Loss: 0.8516, Val Loss: 0.8630
Epoch: 005, Train Loss: 0.8518, Val Loss: 0.8619
Epoch: 006, Train Loss: 0.8501, Val Loss: 0.8619
Epoch: 007, Train Loss: 0.8506, Val Loss: 0.8613
Epoch: 008, Train Loss: 0.8516, Val Loss: 0.8628
Epoch: 009, Train Loss: 0.8509, Val Loss: 0.8631
Epoch: 010, Train Loss: 0.8519, Val Loss: 0.8628
Epoch: 011, Train Loss: 0.8507, Val Loss: 0.8616
Epoch: 012, Train Loss: 0.8507, Val Loss: 0.8614
Epoch: 013, Train Loss: 0.8510, Val Loss: 0.8619
Epoch: 014, Train Loss: 0.8516, Val Loss: 0.8627
Epoch: 015, Train Loss: 0.8511, Val Loss: 0.8620
Epoch: 016, Train Loss: 0.8508, Val Loss: 0.8630
Epoch: 017, Train Loss: 0.8518, Val Loss: 0.8625
Epoch: 018, Train Loss: 0.8508, Val Loss: 0.8624
Epoch: 019, Train Loss: 0.8502, Val Loss: 0.8616
Epoch: 020, Train Lo

In [120]:
test(model=model_gcn2, dataset=data)

Test Accuracy: 0.7373493975903614
Test Precision: 0.3686746987951807
Test Recall: 0.5
Test F1 Score: 0.42441054091539526


C:\Users\Dell\PycharmProjects\Manu_factoid_GCN\.venv\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## FAKE WEIGHTS

In [87]:
weights[1]

tensor(1.9107)

In [88]:
fake_weight=torch.tensor(2.5)

In [89]:
criterion3 = torch.nn.BCEWithLogitsLoss(pos_weight=fake_weight)


In [90]:
train(model=model_gcn, dataset=data,
      criterion=criterion3, optimizer=optimizer,
      epochs=50)

Epoch: 000, Loss: 0.8760
Epoch: 001, Loss: 0.8475
Epoch: 002, Loss: 0.8552
Epoch: 003, Loss: 0.8509
Epoch: 004, Loss: 0.8515
Epoch: 005, Loss: 0.8597
Epoch: 006, Loss: 0.8483
Epoch: 007, Loss: 0.8585
Epoch: 008, Loss: 0.8648
Epoch: 009, Loss: 0.8415
Epoch: 010, Loss: 0.8422
Epoch: 011, Loss: 0.8548
Epoch: 012, Loss: 0.8448
Epoch: 013, Loss: 0.8604
Epoch: 014, Loss: 0.8491
Epoch: 015, Loss: 0.8452
Epoch: 016, Loss: 0.8487
Epoch: 017, Loss: 0.8586
Epoch: 018, Loss: 0.8624
Epoch: 019, Loss: 0.8445
Epoch: 020, Loss: 0.8388
Epoch: 021, Loss: 0.8555
Epoch: 022, Loss: 0.8540
Epoch: 023, Loss: 0.8532
Epoch: 024, Loss: 0.8429
Epoch: 025, Loss: 0.8584
Epoch: 026, Loss: 0.8514
Epoch: 027, Loss: 0.8592
Epoch: 028, Loss: 0.8484
Epoch: 029, Loss: 0.8436
Epoch: 030, Loss: 0.8495
Epoch: 031, Loss: 0.8462
Epoch: 032, Loss: 0.8428
Epoch: 033, Loss: 0.8470
Epoch: 034, Loss: 0.8442
Epoch: 035, Loss: 0.8506
Epoch: 036, Loss: 0.8413
Epoch: 037, Loss: 0.8414
Epoch: 038, Loss: 0.8412
Epoch: 039, Loss: 0.8415


In [50]:
from torch_geometric.utils import to_networkx
import matplotlib.pyplot as plt
import networkx as nx

In [53]:
G = to_networkx(data, to_undirected=True)


In [54]:
nx.write_gexf(G,"graph_data_with.gexf")